In [25]:
## '물리환경 부문' 지표 구성 

# 1) 노후주택 비율
# 2) 신규주택 비율
# 3) 부동산 공시지가

# 소형주택 비율 (제외 결정; 전용면적 산출 불가함)

In [1]:
from geoband.API import *
import pandas as pd

# 1. 물리환경 데이터 불러오기

In [2]:
GetCompasData('SBJ_2208_001', '10', '10_연수구_건축물대장_표제부.xlsx') # 노후주택/신규주택 비율
GetCompasData('SBJ_2208_001', '26', '26_연수구_개별공시지가.xlsx') # 부동산 공시지가

df = pd.read_excel('10_연수구_건축물대장_표제부.xlsx')
dff = pd.read_excel('26_연수구_개별공시지가.xlsx')

[2022-09-20 23:23:23] 성공 - 10_연수구_건축물대장_표제부.xlsx
[2022-09-20 23:23:23] 성공 - 26_연수구_개별공시지가.xlsx


# 2. 데이터 전처리 

## 1) 노후주택/신규주택 비율

In [28]:
# 원본 데이터 확인 및 컬럼 목록 확인 

# local_location    대지위치
# sgg_cd    시군구 코드
# emd_cd    읍면동 코드
# land_gbn    대지구분 코드
# bun    번
# ji    지
# mgm_bldrgst_pk    관리건축물대장PK
# regstr_gb_cd_nm    대장구분명
# bldg_nm    건물 명칭
# dong_nm    동 명칭
# etc_purpose_nm    기타용도명
# hshld_num    세대 수
# use_apr_day    사용승인일

df

,local_location,sgg_cd,emd_cd,land_gbn,bun,ji,mgm_bldrgst_pk,regstr_gb_cd_nm,bldg_nm,dong_nm,etc_purpose_nm,hshld_num,use_apr_day
0,인천광역시 연수구 옥련동 105번지,28185,10100,0,105,0,28185-604,일반,NaN,NaN,주택,0,NaN
1,인천광역시 연수구 옥련동 112-6번지,28185,10100,0,112,6,28185-607,일반,NaN,NaN,단독주택,1,19720803
2,인천광역시 연수구 옥련동 115-12번지,28185,10100,0,115,12,28185-100179967,일반,NaN,NaN,"단독주택, 제1,2종근린생활시설",0,20091014
3,인천광역시 연수구 옥련동 116-10번지,28185,10100,0,116,10,28185-609,일반,NaN,NaN,단독주택,0,19850202
4,인천광역시 연수구 옥련동 116-11번지,28185,10100,0,116,11,28185-100180087,일반,NaN,NaN,다가구주택,0,20091109
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9230,인천광역시 연수구 송도동 93번지,28185,10600,0,93,0,28185-100179250,일반,NaN,인스타 Ⅱ,판매시설,0,20090731
9231,인천광역시 연수구 송도동 96번지,28185,10600,0,96,0,28185-100284941,집합,리치센트럴,리치센트럴,"자동차관련시설(주차장),근린생활시설,판매시설",0,20220428
9232,인천광역시 연수구 송도동 98-1번지,28185,10600,0,98,1,28185-100284380,집합,닥터플러스몰,닥터플러스몰,"판매시설,제1,2종근린생활시설,업무시설",0,20220207
9233,인천광역시 연수구 송도동 98-2번지,28185,10600,0,98,2,28185-100232355,집합,송도아라플라자,NaN,"판매시설, 근린생활시설",0,20171031


In [29]:
# 중요 컬럼(사용승인일, 대지위치, 기타용도명)이 null 값인 데이터 행 삭제  
df = df.dropna(axis=0, subset=['use_apr_day', 'local_location', 'etc_purpose_nm'])

# 불필요한 컬럼 삭제 
df.drop(['sgg_cd', 'emd_cd', 'land_gbn', 'bun', 'ji', 'regstr_gb_cd_nm', 'bldg_nm', 'dong_nm', 'hshld_num'], axis=1, inplace=True)

# 인덱스 재정렬 및 데이터프레임 확인 
df.reset_index(drop=True, inplace=True)
print(df.info())

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8915 entries, 0 to 8914
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  8915 non-null   object
 1   mgm_bldrgst_pk  8915 non-null   object
 2   etc_purpose_nm  8915 non-null   object
 3   use_apr_day     8915 non-null   object
dtypes: object(4)
memory usage: 278.7+ KB
None


/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,local_location,mgm_bldrgst_pk,etc_purpose_nm,use_apr_day
0,인천광역시 연수구 옥련동 112-6번지,28185-607,단독주택,19720803
1,인천광역시 연수구 옥련동 115-12번지,28185-100179967,"단독주택, 제1,2종근린생활시설",20091014
2,인천광역시 연수구 옥련동 116-10번지,28185-609,단독주택,19850202
3,인천광역시 연수구 옥련동 116-11번지,28185-100180087,다가구주택,20091109
4,인천광역시 연수구 옥련동 116-12번지,28185-610,"다가구주택, 근린생활시설",20000214
...,...,...,...,...
8910,인천광역시 연수구 송도동 93번지,28185-100179250,판매시설,20090731
8911,인천광역시 연수구 송도동 96번지,28185-100284941,"자동차관련시설(주차장),근린생활시설,판매시설",20220428
8912,인천광역시 연수구 송도동 98-1번지,28185-100284380,"판매시설,제1,2종근린생활시설,업무시설",20220207
8913,인천광역시 연수구 송도동 98-2번지,28185-100232355,"판매시설, 근린생활시설",20171031


In [30]:
# 'local_location' 컬럼: 시구동 -> 동 변경  
df['local_location'] = df['local_location'].str.split(' ').str[2]

# 'use_apr_day' 컬럼: 연도일자 -> 연도 변경 
df['use_apr_day'] = df['use_apr_day'].str[:4]

# 'use_apr_day' 컬럼: object -> int64 타입 변경 
df = df.astype({'use_apr_day' : 'int64'})

df

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,local_location,mgm_bldrgst_pk,etc_purpose_nm,use_apr_day
0,옥련동,28185-607,단독주택,1972
1,옥련동,28185-100179967,"단독주택, 제1,2종근린생활시설",2009
2,옥련동,28185-609,단독주택,1985
3,옥련동,28185-100180087,다가구주택,2009
4,옥련동,28185-610,"다가구주택, 근린생활시설",2000
...,...,...,...,...
8910,송도동,28185-100179250,판매시설,2009
8911,송도동,28185-100284941,"자동차관련시설(주차장),근린생활시설,판매시설",2022
8912,송도동,28185-100284380,"판매시설,제1,2종근린생활시설,업무시설",2022
8913,송도동,28185-100232355,"판매시설, 근린생활시설",2017


In [31]:
# 중복되는 '관리건축물대장PK'이 있는지 확인
df['mgm_bldrgst_pk'].nunique()

# 관리건축물대장PK'개수: 8915, 전체 데이터 행 개수: 8915 -> 즉, 중복 없음을 확인. 

8915

In [32]:
# 기타용도('etc_purpose_nm') 구성 확인
print(df['etc_purpose_nm'].nunique())
df['etc_purpose_nm'].unique()

1123


array(['단독주택', '단독주택, 제1,2종근린생활시설', '다가구주택', ...,
       '자동차관련시설(주차장),근린생활시설,판매시설', '판매시설,제1,2종근린생활시설,업무시설', '판매시설(대형점)'],
      dtype=object)

In [33]:
# n = 40
# df['etc_purpose_nm'].value_counts()[:n].index.tolist()
# df[['etc_purpose_nm']].groupby('etc_purpose_nm')['etc_purpose_nm'].count().nlargest(60)
# df[df['etc_purpose_nm'].str.contains('주택|아파트|오피스텔|연립|다세대|다가구|단독')]['etc_purpose_nm'].nunique()

In [34]:
# 다양한 건축물 가운데 주택만 추출하여 데이터 프레임 재구조화 
df_housing = df[df['etc_purpose_nm'].str.contains('주택|아파트|오피스텔|연립|다세대|다가구|단독')]
df_housing.reset_index(drop=True, inplace=True)
print(df_housing.info())
df_housing

# (참고) 주택의 범위를 어디까지 볼 것인가? 
# 단독주택, 다가구, 영업겸용 단독주택, (작성중)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5892 entries, 0 to 5891
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  5892 non-null   object
 1   mgm_bldrgst_pk  5892 non-null   object
 2   etc_purpose_nm  5892 non-null   object
 3   use_apr_day     5892 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 184.2+ KB
None


,local_location,mgm_bldrgst_pk,etc_purpose_nm,use_apr_day
0,옥련동,28185-607,단독주택,1972
1,옥련동,28185-100179967,"단독주택, 제1,2종근린생활시설",2009
2,옥련동,28185-609,단독주택,1985
3,옥련동,28185-100180087,다가구주택,2009
4,옥련동,28185-610,"다가구주택, 근린생활시설",2000
...,...,...,...,...
5887,송도동,28185-100184099,아파트,2010
5888,송도동,28185-100183984,아파트,2010
5889,송도동,28185-100184030,아파트,2010
5890,송도동,28185-100184005,아파트,2010


In [35]:
df_housing['local_location'].unique()

array(['옥련동', '선학동', '연수동', '청학동', '동춘동', '송도동'], dtype=object)

In [36]:
# 옥련동 주택 데이터 
okryun = df_housing[df_housing['local_location'] == '옥련동']
okryun.reset_index(drop=True, inplace=True)
print(okryun.info())
             
#  옥련동 20년 이상 된 주택 데이터
okryun_old = okryun[okryun['use_apr_day'] <= 2002]
okryun_old.reset_index(drop=True, inplace=True) 
print(okryun_old.info())   

# 옥련동 5년 안에 신축된 주택 데이터
okryun_new = okryun[okryun['use_apr_day'] >= 2017]
okryun_new.reset_index(drop=True, inplace=True) 
print(okryun_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139 entries, 0 to 1138
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  1139 non-null   object
 1   mgm_bldrgst_pk  1139 non-null   object
 2   etc_purpose_nm  1139 non-null   object
 3   use_apr_day     1139 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 35.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  944 non-null    object
 1   mgm_bldrgst_pk  944 non-null    object
 2   etc_purpose_nm  944 non-null    object
 3   use_apr_day     944 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 29.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
--- 

In [37]:
# 선학동 

seonhak = df_housing[df_housing['local_location'] == '선학동']
seonhak.reset_index(drop=True, inplace=True)
print(seonhak.info())
             
seonhak_old = seonhak[seonhak['use_apr_day'] <= 2002]
seonhak_old.reset_index(drop=True, inplace=True) 
print(seonhak_old .info())  

seonhak_new = seonhak[seonhak['use_apr_day'] >= 2017]
seonhak_new.reset_index(drop=True, inplace=True) 
print(seonhak_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  495 non-null    object
 1   mgm_bldrgst_pk  495 non-null    object
 2   etc_purpose_nm  495 non-null    object
 3   use_apr_day     495 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 15.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  438 non-null    object
 1   mgm_bldrgst_pk  438 non-null    object
 2   etc_purpose_nm  438 non-null    object
 3   use_apr_day     438 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 13.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  -

In [38]:
# 연수동 

yeonsu = df_housing[df_housing['local_location'] == '연수동']
yeonsu.reset_index(drop=True, inplace=True)
print(yeonsu.info())
             
yeonsu_old = yeonsu[yeonsu['use_apr_day'] <= 2002]
yeonsu_old.reset_index(drop=True, inplace=True) 
print(yeonsu_old .info())  

yeonsu_new = yeonsu[yeonsu['use_apr_day'] >= 2017]
yeonsu_new.reset_index(drop=True, inplace=True) 
print(yeonsu_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  1614 non-null   object
 1   mgm_bldrgst_pk  1614 non-null   object
 2   etc_purpose_nm  1614 non-null   object
 3   use_apr_day     1614 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 50.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544 entries, 0 to 1543
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  1544 non-null   object
 1   mgm_bldrgst_pk  1544 non-null   object
 2   etc_purpose_nm  1544 non-null   object
 3   use_apr_day     1544 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 48.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
--

In [39]:
# 청학동 

cheonghak = df_housing[df_housing['local_location'] == '청학동']
cheonghak.reset_index(drop=True, inplace=True)
print(cheonghak.info())
             
cheonghak_old = cheonghak[cheonghak['use_apr_day'] <= 2002]
cheonghak_old.reset_index(drop=True, inplace=True) 
print(cheonghak_old .info())  

cheonghak_new = cheonghak[cheonghak['use_apr_day'] >= 2017]
cheonghak_new.reset_index(drop=True, inplace=True) 
print(cheonghak_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  1311 non-null   object
 1   mgm_bldrgst_pk  1311 non-null   object
 2   etc_purpose_nm  1311 non-null   object
 3   use_apr_day     1311 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 41.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1214 entries, 0 to 1213
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  1214 non-null   object
 1   mgm_bldrgst_pk  1214 non-null   object
 2   etc_purpose_nm  1214 non-null   object
 3   use_apr_day     1214 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 38.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
--

In [40]:
# 동춘동 

dongchun = df_housing[df_housing['local_location'] == '동춘동']
dongchun.reset_index(drop=True, inplace=True)
print(dongchun.info())
             
dongchun_old = dongchun[dongchun['use_apr_day'] <= 2002]
dongchun_old.reset_index(drop=True, inplace=True) 
print(dongchun_old.info())

dongchun_new = dongchun[dongchun['use_apr_day'] >= 2017]
dongchun_new.reset_index(drop=True, inplace=True) 
print(dongchun_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  556 non-null    object
 1   mgm_bldrgst_pk  556 non-null    object
 2   etc_purpose_nm  556 non-null    object
 3   use_apr_day     556 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 17.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  432 non-null    object
 1   mgm_bldrgst_pk  432 non-null    object
 2   etc_purpose_nm  432 non-null    object
 3   use_apr_day     432 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 13.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  -

In [41]:
# 송도동 

songdo = df_housing[df_housing['local_location'] == '송도동']
songdo.reset_index(drop=True, inplace=True)
print(songdo.info())
             
songdo_old = songdo[songdo['use_apr_day'] <= 2002]
songdo_old.reset_index(drop=True, inplace=True) 
print(songdo_old.info())

songdo_new = songdo[songdo['use_apr_day'] >= 2017]
songdo_new.reset_index(drop=True, inplace=True) 
print(songdo_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  777 non-null    object
 1   mgm_bldrgst_pk  777 non-null    object
 2   etc_purpose_nm  777 non-null    object
 3   use_apr_day     777 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 24.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   local_location  0 non-null      object
 1   mgm_bldrgst_pk  0 non-null      object
 2   etc_purpose_nm  0 non-null      object
 3   use_apr_day     0 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 124.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------ 

In [42]:
print(f'옥련동 노후주택 비율: {944/1139*100}')
print(f'선학동 노후주택 비율: {438/495*100}')
print(f'연수동 노후주택 비율: {1544/1614*100}')
print(f'청학동 노후주택 비율: {1214/1311*100}')
print(f'동춘동 노후주택 비율: {432/556*100}')
print(f'송도동 노후주택 비율: {0/777*100}')

옥련동 노후주택 비율: 82.87971905179982
선학동 노후주택 비율: 88.48484848484848
연수동 노후주택 비율: 95.6629491945477
청학동 노후주택 비율: 92.60106788710908
동춘동 노후주택 비율: 77.6978417266187
송도동 노후주택 비율: 0.0


In [43]:
print(f'옥련동 신규주택 비율: {64/1139*100}')
print(f'선학동 신규주택 비율: {17/495*100}')
print(f'연수동 신규주택 비율: {55/1614*100}')
print(f'청학동 신규주택 비율: {19/1311*100}')
print(f'동춘동 신규주택 비율: {82/556*100}')
print(f'송도동 신규주택 비율: {208/777*100}')

옥련동 신규주택 비율: 5.618964003511852
선학동 신규주택 비율: 3.4343434343434343
연수동 신규주택 비율: 3.407682775712516
청학동 신규주택 비율: 1.4492753623188406
동춘동 신규주택 비율: 14.748201438848922
송도동 신규주택 비율: 26.76962676962677


In [44]:
# '노후주택 비율, 신규주택 비율' 최종 데이터

housing_old_new = pd.DataFrame( [ ['옥련동',len(okryun), len(okryun_old), len(okryun_new), (len(okryun_old)/len(okryun)*100), (len(okryun_new)/len(okryun)*100)] ,
              ['선학동',len(seonhak), len(seonhak_old), len(seonhak_new), (len(seonhak_old)/len(seonhak)*100), (len(seonhak_new)/len(seonhak)*100)],
              ['연수동',len(yeonsu), len(yeonsu_old), len(yeonsu_new), (len(yeonsu_old)/len(yeonsu)*100), (len(yeonsu_new)/len(yeonsu)*100)],
              ['청학동',len(cheonghak), len(cheonghak_old), len(cheonghak_new), (len(cheonghak_old)/len(cheonghak)*100), (len(cheonghak_new)/len(cheonghak)*100)],
              ['동춘동',len(dongchun), len(dongchun_old), len(dongchun_new), (len(dongchun_old)/len(dongchun)*100), (len(dongchun_new)/len(dongchun)*100)],
              ['송도동',len(songdo), len(songdo_old), len(songdo_new), (len(songdo_old)/len(songdo)*100), (len(songdo_new)/len(songdo)*100)]])

housing_old_new.columns=['dong','total', 'over_20_years', 'within_5_years', 'over_20_years_rate','within_5_years_rate']

housing_old_new

,dong,total,over_20_years,within_5_years,over_20_years_rate,within_5_years_rate
0,옥련동,1139,944,64,82.879719,5.618964
1,선학동,495,438,17,88.484848,3.434343
2,연수동,1614,1544,55,95.662949,3.407683
3,청학동,1311,1214,19,92.601068,1.449275
4,동춘동,556,432,82,77.697842,14.748201
5,송도동,777,0,208,0.000000,26.769627


In [45]:
# 최종 데이터 내보내기 
housing_old_new.to_csv('최종_물리환경_노후주택_신규주택 비율.csv')

## 2) 부동산 공시지가

In [3]:
dff = pd.read_excel('26_연수구_개별공시지가.xlsx')
dff

,unique_no,bjd_cd,bjd_nm,s_land_cd,s_land_nm,jibun,year,month,land_value,office_date
0,2818510100100010000,2818510100,인천광역시 연수구 옥련동,1,일반,1,2013,1,44300,2013-05-31
1,2818510100100010000,2818510100,인천광역시 연수구 옥련동,1,일반,1,2014,1,45300,2014-05-30
2,2818510100100010001,2818510100,인천광역시 연수구 옥련동,1,일반,1-1,2013,1,44300,2013-05-31
3,2818510100100010001,2818510100,인천광역시 연수구 옥련동,1,일반,1-1,2014,1,45300,2014-05-30
4,2818510100100020000,2818510100,인천광역시 연수구 옥련동,1,일반,2,2013,1,183000,2013-05-31
...,...,...,...,...,...,...,...,...,...,...
120736,2818510600200010001,2818510600,인천광역시 연수구 송도동,2,산,1-1,2017,1,285000,2017-05-31
120737,2818510600200010001,2818510600,인천광역시 연수구 송도동,2,산,1-1,2018,1,295000,2018-05-31
120738,2818510600200010001,2818510600,인천광역시 연수구 송도동,2,산,1-1,2019,1,306000,2019-05-31
120739,2818510600200010001,2818510600,인천광역시 연수구 송도동,2,산,1-1,2020,1,318000,2020-05-29


In [4]:
# 기준연도 확인하기 
dff['year'].unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [5]:
# 고유번호 확인하기 
dff['unique_no'].nunique()

# 확인해 본 결과, 총 데이터 개수는 120741개인데, 고유번호 개수는 15251개. 
# 이 의미는 연도별로 개별공시지가가 산출된 건축물(땅)이 있다는 것으로 해석 가능.   

15251

In [6]:
# 컬럼 정리하기 (불필요한 컬럼 제거, 컬럼의 값을 간단히 (예. 인천광역시 연수구 송도동 -> 송도동))

dff.drop(['bjd_cd','s_land_cd', 's_land_nm', 'jibun', 'office_date'], axis=1, inplace=True)
dff['bjd_nm'] = dff['bjd_nm'].str.split(' ').str[2]
dff

,unique_no,bjd_nm,year,month,land_value
0,2818510100100010000,옥련동,2013,1,44300
1,2818510100100010000,옥련동,2014,1,45300
2,2818510100100010001,옥련동,2013,1,44300
3,2818510100100010001,옥련동,2014,1,45300
4,2818510100100020000,옥련동,2013,1,183000
...,...,...,...,...,...
120736,2818510600200010001,송도동,2017,1,285000
120737,2818510600200010001,송도동,2018,1,295000
120738,2818510600200010001,송도동,2019,1,306000
120739,2818510600200010001,송도동,2020,1,318000


In [7]:
# 동, 연도에 따른 개별공시지가 평균 
pd.options.display.float_format = '{:.2f}'.format
dff_pivoted = dff.pivot_table(index = 'bjd_nm', columns='year',values='land_value', aggfunc='mean').reset_index().rename_axis(None, axis=1)
dff_pivoted

,bjd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,동춘동,639680.88,655799.06,674709.79,716726.90,735328.46,775838.93,810353.69,847901.50,895940.95
1,선학동,477784.73,518313.72,544123.30,556419.81,573275.47,591717.84,626677.49,636446.58,696877.60
2,송도동,1241828.10,1250364.34,1252165.53,1260555.34,1189278.62,1222652.95,1279880.35,1335322.15,1463494.72
3,연수동,932946.58,962319.43,999228.48,1027040.20,1069174.32,1100425.01,1159428.69,1214546.21,1355893.24
4,옥련동,727422.82,742250.45,784857.91,807322.34,836111.03,869171.54,919594.29,954249.45,1034101.37
5,청학동,695190.33,720857.22,742402.97,759817.40,785352.47,820860.98,861773.18,919328.04,1029153.78


In [8]:
# 컬럼면 변경 bjd_nm -> dong

dff_pivoted.rename(columns={'bjd_nm':'dong'}, inplace=True)
dff_pivoted

,dong,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,동춘동,639680.88,655799.06,674709.79,716726.90,735328.46,775838.93,810353.69,847901.50,895940.95
1,선학동,477784.73,518313.72,544123.30,556419.81,573275.47,591717.84,626677.49,636446.58,696877.60
2,송도동,1241828.10,1250364.34,1252165.53,1260555.34,1189278.62,1222652.95,1279880.35,1335322.15,1463494.72
3,연수동,932946.58,962319.43,999228.48,1027040.20,1069174.32,1100425.01,1159428.69,1214546.21,1355893.24
4,옥련동,727422.82,742250.45,784857.91,807322.34,836111.03,869171.54,919594.29,954249.45,1034101.37
5,청학동,695190.33,720857.22,742402.97,759817.40,785352.47,820860.98,861773.18,919328.04,1029153.78


In [9]:
dff_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   dong    6 non-null      object 
 1   2013    6 non-null      float64
 2   2014    6 non-null      float64
 3   2015    6 non-null      float64
 4   2016    6 non-null      float64
 5   2017    6 non-null      float64
 6   2018    6 non-null      float64
 7   2019    6 non-null      float64
 8   2020    6 non-null      float64
 9   2021    6 non-null      float64
dtypes: float64(9), object(1)
memory usage: 608.0+ bytes


In [10]:
# 데이터 내보내기 
dff_pivoted.to_csv('최종_물리환경_부동산공시지가_2013-2021.csv')

In [54]:
## 아래는 그냥 참고 
## 연도별로 데이터 프레임 생성
# dff2013 = dff[dff['year']== 2013].reset_index(drop=True)
# dff2014 = dff[dff['year']== 2014].reset_index(drop=True)
# dff2015 = dff[dff['year']== 2015].reset_index(drop=True)
# dff2016 = dff[dff['year']== 2016].reset_index(drop=True)
# dff2017 = dff[dff['year']== 2017].reset_index(drop=True)
# dff2018 = dff[dff['year']== 2018].reset_index(drop=True)
# dff2019 = dff[dff['year']== 2019].reset_index(drop=True)
# dff2020 = dff[dff['year']== 2020].reset_index(drop=True)
# dff2021 = dff[dff['year']== 2021].reset_index(drop=True)

In [55]:
# # 월('month') 컬럼 삭제 가능 여부 살펴보고자, 아래와 같이 '고유번호('unique_no')와 데이터 행 개수 일치 여부 확인

# print(dff2013.info())
# print(dff2013['unique_no'].nunique())
# print('-----------------------------')

# print(dff2014.info())
# print(dff2014['unique_no'].nunique())
# print('-----------------------------')

# print(dff2015.info())
# print(dff2015['unique_no'].nunique())
# print('-----------------------------')

# print(dff2016.info())
# print(dff2016['unique_no'].nunique())
# print('-----------------------------')

# print(dff2017.info())
# print(dff2017['unique_no'].nunique())
# print('-----------------------------')

# print(dff2018.info())
# print(dff2018['unique_no'].nunique())
# print('-----------------------------')

# print(dff2019.info())
# print(dff2019['unique_no'].nunique())
# print('-----------------------------')

# print(dff2019.info())
# print(dff2019['unique_no'].nunique())
# print('-----------------------------')

# print(dff2019.info())
# print(dff2019['unique_no'].nunique())
# print('-----------------------------')

# print(dff2020.info())
# print(dff2020['unique_no'].nunique())
# print('-----------------------------')

# print(dff2021.info())
# print(dff2021['unique_no'].nunique())
# print('-----------------------------')